In [1]:
# ===============================================================
# ML Project — Macro + Market Data
# Pipeline : FRED + WorldBank + OCDE(KEI/DB.NOMICS) → Merge → Weekly → GB
# Output files : CSV
# ===============================================================

# =========================
# 0) Data Import
# =========================
import re, datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import wbdata
import requests
import yfinance as yf
from pathlib import Path

START_DATE = "2010-01-01"
END_DATE   = "2025-09-01"
date_index = pd.date_range(start=START_DATE, end=END_DATE, freq="D")

OUT = Path("../data")
OUT.mkdir(parents=True, exist_ok=True)

In [2]:
# ==============================
# 1) FRED — Times Series
# ==============================

fred_series = {
    "CPIAUCSL": "Inflation_CPI",
    "UNRATE": "Unemployment_Rate",
    "FEDFUNDS": "Fed_Funds_Rate",
    "GDPC1": "Real_GDP",
    "T10Y3M": "Yield_Curve_Spread",
    "INDPRO": "Industrial_Production",
    "M2SL": "M2_Money_Supply",
    "PCE": "Personal_Consumption_Expenditures",
    "RECPROUSM156N": "Recession_Probability",
}
fred_data = pdr.DataReader(list(fred_series.keys()), "fred", start=START_DATE, end=END_DATE)
fred_data.rename(columns=fred_series, inplace=True)
fred_data = fred_data.sort_index().reindex(date_index).ffill().bfill()

from IPython.display import display
print("FRED shape:", fred_data.shape)
display(fred_data.head())

FRED shape: (5723, 9)


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.71,3.70,89.3426,8477.7,10056.1,0.08


In [3]:
from IPython.display import display
pd.set_option("display.max_rows", 200)   # optionnel

print("FRED 2010 Jan–Mar:")
display(fred_data.loc["2010-01":"2010-03"])

print("FRED — 50 premières lignes:")
display(fred_data.iloc[:50])

print("FRED — 50 dernières lignes:")
display(fred_data.iloc[-50:])

FRED 2010 Jan–Mar:


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.71,3.70,89.3426,8477.7,10056.1,0.08
2010-01-06,217.488,9.8,0.11,16582.71,3.79,89.3426,8477.7,10056.1,0.08
2010-01-07,217.488,9.8,0.11,16582.71,3.80,89.3426,8477.7,10056.1,0.08
2010-01-08,217.488,9.8,0.11,16582.71,3.78,89.3426,8477.7,10056.1,0.08
2010-01-09,217.488,9.8,0.11,16582.71,3.78,89.3426,8477.7,10056.1,0.08
2010-01-10,217.488,9.8,0.11,16582.71,3.78,89.3426,8477.7,10056.1,0.08


FRED — 50 premières lignes:


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.71,3.77,89.3426,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.71,3.70,89.3426,8477.7,10056.1,0.08
2010-01-06,217.488,9.8,0.11,16582.71,3.79,89.3426,8477.7,10056.1,0.08
2010-01-07,217.488,9.8,0.11,16582.71,3.80,89.3426,8477.7,10056.1,0.08
2010-01-08,217.488,9.8,0.11,16582.71,3.78,89.3426,8477.7,10056.1,0.08
2010-01-09,217.488,9.8,0.11,16582.71,3.78,89.3426,8477.7,10056.1,0.08
2010-01-10,217.488,9.8,0.11,16582.71,3.78,89.3426,8477.7,10056.1,0.08


FRED — 50 dernières lignes:


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2025-07-14,322.132,4.2,4.33,23770.976,0.01,101.5910,22028.8,20981.2,0.64
2025-07-15,322.132,4.2,4.33,23770.976,0.08,101.5910,22028.8,20981.2,0.64
2025-07-16,322.132,4.2,4.33,23770.976,0.05,101.5910,22028.8,20981.2,0.64
2025-07-17,322.132,4.2,4.33,23770.976,0.06,101.5910,22028.8,20981.2,0.64
2025-07-18,322.132,4.2,4.33,23770.976,0.04,101.5910,22028.8,20981.2,0.64
2025-07-19,322.132,4.2,4.33,23770.976,0.04,101.5910,22028.8,20981.2,0.64
2025-07-20,322.132,4.2,4.33,23770.976,0.04,101.5910,22028.8,20981.2,0.64
2025-07-21,322.132,4.2,4.33,23770.976,-0.03,101.5910,22028.8,20981.2,0.64
2025-07-22,322.132,4.2,4.33,23770.976,-0.06,101.5910,22028.8,20981.2,0.64
2025-07-23,322.132,4.2,4.33,23770.976,-0.01,101.5910,22028.8,20981.2,0.64


In [4]:
# ==============================
# 2) WORLD BANK — USA
# ==============================
wb_indicators = {
    "NY.GDP.MKTP.CD": "GDP_Current_USD",
    "FP.CPI.TOTL.ZG": "Inflation_Annual_Pct",
    "SL.UEM.TOTL.ZS": "Unemployment_Total_Pct",
    "NE.EXP.GNFS.ZS": "Exports_GDP_Pct",
    "NE.IMP.GNFS.ZS": "Imports_GDP_Pct",
    "GC.DOD.TOTL.GD.ZS": "Public_Debt_GDP_Pct",
}
try:
    wb_data = wbdata.get_dataframe(
        wb_indicators, country="US",
        date=(dt.datetime(2010,1,1), dt.datetime(2025,9,1))
    )
except TypeError:
    wb_data = wbdata.get_dataframe(
        wb_indicators, country="US",
        data_date=(dt.datetime(2010,1,1), dt.datetime(2025,9,1))
    )
wb_data.index = pd.to_datetime(wb_data.index, errors="coerce")
wb_data = wb_data.sort_index().reindex(date_index).ffill().bfill()

print("World Bank shape:", wb_data.shape)
display(wb_data.head())


World Bank shape: (5723, 6)


,GDP_Current_USD,Inflation_Annual_Pct,Unemployment_Total_Pct,Exports_GDP_Pct,Imports_GDP_Pct,Public_Debt_GDP_Pct
2010-01-01,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-02,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-03,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-04,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-05,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374


In [5]:
from IPython.display import display
print("World Bank — 2025:")
display(wb_data.loc["2025"])

World Bank — 2025:


,GDP_Current_USD,Inflation_Annual_Pct,Unemployment_Total_Pct,Exports_GDP_Pct,Imports_GDP_Pct,Public_Debt_GDP_Pct
2025-01-01,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-02,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-03,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-04,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-05,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
...,...,...,...,...,...,...
2025-08-28,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-29,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-30,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-31,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553


In [6]:
# =================================================
# 3) OCDE (KEI) via DB.NOMICS — CPI & Unemployment
# =================================================
def oecd_kei_to_wide(series_ids):
    base = "https://api.db.nomics.world/v22/series?observations=1&format=csv&series_ids="
    url = base + ",".join(series_ids)
    df = pd.read_csv(url, low_memory=False)

    # trouver colonne période
    pcol = next((c for c in df.columns if c.strip().lower() in {"period","time","time_period","date"}), None)
    if pcol is None:
        raise KeyError(f"Colonne période introuvable: {list(df.columns)[:10]}")
    df = df.rename(columns={pcol: "period"})

    # renommer colonnes → extraire ID entre parenthèses (...(OECD/KEI/...))
    ren = {}
    for c in df.columns:
        if c == "period": continue
        m = re.search(r"\(([^()]+)\)\s*$", str(c))
        ren[c] = (m.group(1) if m else c).upper()
    df = df.rename(columns=ren)

    keep = ["period"] + [s.upper() for s in series_ids]
    missing = [s for s in series_ids if s.upper() not in df.columns]
    if missing:
        # On laisse passer, on créera un fallback
        print("⚠️ Séries OCDE manquantes dans le CSV:", missing)
        keep = [k for k in keep if k in df.columns]
    wide = df[keep].copy()

    # index = fin de mois
    wide.index = pd.PeriodIndex(wide["period"].astype(str), freq="M").end_time
    wide = wide.drop(columns=["period"]).sort_index()
    wide.index.name = "DATE"
    return wide

oecd_series_ids = [
    "OECD/KEI/CPALTT01.USA.ST.M",  # CPI 2015=100, mensuel, s.a.
    "OECD/KEI/LRHUTTTT.USA.ST.M",  # Unemployment %, mensuel, s.a.
]
oecd_data = oecd_kei_to_wide(oecd_series_ids)
oecd_data = oecd_data.reindex(date_index).ffill().bfill()
oecd_data = oecd_data.rename(columns={
    "OECD/KEI/CPALTT01.USA.ST.M": "OECD_CPI_2015idx_USA",
    "OECD/KEI/LRHUTTTT.USA.ST.M": "OECD_Unemp_rate_pct_USA",
})
print("OCDE shape (avant fallback):", oecd_data.shape, "| NA:", int(oecd_data.isna().sum().sum()))


OCDE shape (avant fallback): (5723, 2) | NA: 11446


In [7]:
# ===========================================================
# 3bis) Fallback OCDE depuis FRED si colonnes vides (0 NaN)
# ===========================================================
def fill_oecd_from_fred(oecd_df, fred_df, date_index):
    o = oecd_df.copy()
    changed = []

    # CPI indice 2015=100 à partir de CPIAUCSL (mensuel)
    if "OECD_CPI_2015idx_USA" in o.columns and o["OECD_CPI_2015idx_USA"].isna().all():
        cpi_m = fred_df["Inflation_CPI"].resample("ME").last()
        base_2015 = cpi_m.loc["2015-01-31":"2015-12-31"].mean()
        o["OECD_CPI_2015idx_USA"] = (cpi_m/base_2015*100).reindex(date_index).ffill().bfill()
        changed.append("OECD_CPI_2015idx_USA ← FRED CPIAUCSL (base 2015=100)")

    # Unemployment % à partir de UNRATE (mensuel)
    if "OECD_Unemp_rate_pct_USA" in o.columns and o["OECD_Unemp_rate_pct_USA"].isna().all():
        un_m = fred_df["Unemployment_Rate"].resample("ME").last()
        o["OECD_Unemp_rate_pct_USA"] = un_m.reindex(date_index).ffill().bfill()
        changed.append("OECD_Unemp_rate_pct_USA ← FRED UNRATE")

    if changed:
        print("OCDE fallback:", changed)
    else:
        print("OCDE fallback: aucun")
    return o

oecd_data = fill_oecd_from_fred(oecd_data, fred_data, date_index)
print("OCDE NA après fallback:", int(oecd_data.isna().sum().sum()))


OCDE fallback: ['OECD_CPI_2015idx_USA ← FRED CPIAUCSL (base 2015=100)', 'OECD_Unemp_rate_pct_USA ← FRED UNRATE']
OCDE NA après fallback: 0


In [8]:
oecd_data

,OECD_CPI_2015idx_USA,OECD_Unemp_rate_pct_USA
2010-01-01,91.766411,9.8
2010-01-02,91.766411,9.8
2010-01-03,91.766411,9.8
2010-01-04,91.766411,9.8
2010-01-05,91.766411,9.8
...,...,...
2025-08-28,135.919671,4.2
2025-08-29,135.919671,4.2
2025-08-30,135.919671,4.2
2025-08-31,136.439499,4.3


In [9]:
fred_data

,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.710,3.77,89.3426,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.710,3.77,89.3426,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.710,3.77,89.3426,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.710,3.77,89.3426,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.710,3.70,89.3426,8477.7,10056.1,0.08
...,...,...,...,...,...,...,...,...,...
2025-08-28,323.364,4.3,4.33,23770.976,-0.04,101.3300,22108.4,21087.1,0.96
2025-08-29,323.364,4.3,4.33,23770.976,0.00,101.3300,22108.4,21087.1,0.96
2025-08-30,323.364,4.3,4.33,23770.976,0.00,101.3300,22108.4,21087.1,0.96
2025-08-31,323.364,4.3,4.33,23770.976,0.00,101.3300,22108.4,21087.1,0.96


In [10]:
wb_data

,GDP_Current_USD,Inflation_Annual_Pct,Unemployment_Total_Pct,Exports_GDP_Pct,Imports_GDP_Pct,Public_Debt_GDP_Pct
2010-01-01,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-02,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-03,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-04,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-05,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
...,...,...,...,...,...,...
2025-08-28,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-29,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-30,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-31,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553


In [11]:
oecd_data

,OECD_CPI_2015idx_USA,OECD_Unemp_rate_pct_USA
2010-01-01,91.766411,9.8
2010-01-02,91.766411,9.8
2010-01-03,91.766411,9.8
2010-01-04,91.766411,9.8
2010-01-05,91.766411,9.8
...,...,...
2025-08-28,135.919671,4.2
2025-08-29,135.919671,4.2
2025-08-30,135.919671,4.2
2025-08-31,136.439499,4.3


In [12]:
# -----------------------------------------------
# 4) MERGE ALL MACRO DATA Sources (+ saving)
# -----------------------------------------------
OUT = Path("../data")
OUT.mkdir(parents=True, exist_ok=True)

macro_data = (
    fred_data
    .join(wb_data, how="outer")
    .join(oecd_data, how="outer")
    .sort_index()
    .ffill()
    .bfill()
)

print("NA total (macro):", int(macro_data.isna().sum().sum()))

# Save files in /data/
macro_data.to_csv(OUT / "Macro_AllSources.csv")
fred_data.to_csv(OUT / "Macro_FRED.csv")
wb_data.to_csv(OUT / "Macro_WorldBank.csv")
oecd_data.to_csv(OUT / "Macro_OECD.csv")

print(f"Saved macro CSVs to: {OUT.resolve()}")

NA total (macro): 0
Saved macro CSVs to: C:\Users\dax_a\Documents\GitHub\ESILV-MLproject-AU-BEJOT\data


In [14]:
#
# 5) Market Data 
#

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# --- PARAMETERS ---
START_DATE = "2010-01-01"
END_DATE   = datetime.today().strftime("%Y-%m-%d")

# Utility function to fetch data
def fetch_data(tickers, start=START_DATE, end=None):
    data = yf.download(tickers, start=start, end=end, progress=False, group_by='ticker', auto_adjust=True)

    # Handle structure depending on number of tickers
    if isinstance(data.columns, pd.MultiIndex):
        # MultiIndex when multiple tickers are requested
        data_close = pd.DataFrame()
        for t in tickers:
            # Try to extract 'Adj Close' or fall back to 'Close'
            if ('Adj Close' in data[t].columns):
                data_close[t] = data[t]['Adj Close']
            elif ('Close' in data[t].columns):
                data_close[t] = data[t]['Close']
            else:
                print(f"⚠️ No valid price data found for {t}")
    else:
        # Single ticker case
        col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'
        data_close = data[[col]].rename(columns={col: tickers[0]})

    data_close.dropna(how='all', inplace=True)
    return data_close

In [15]:
index_tickers = {
    "S&P500": "^GSPC",
    "NASDAQ": "^IXIC",
    "DowJones": "^DJI",
    "CAC40": "^FCHI",
    "DAX": "^GDAXI",
    "FTSE100": "^FTSE",
    "Nikkei225": "^N225",
    "HangSeng": "^HSI",
    "MSCIWorld": "URTH"
}

indexes = fetch_data(list(index_tickers.values()))
indexes.columns = index_tickers.keys()
print(indexes.head())
print(indexes.tail())

                 S&P500       NASDAQ      DowJones        CAC40          DAX  \
Date                                                                           
2010-01-04  1132.989990  2308.419922  10583.959961  4013.969971  6048.299805   
2010-01-05  1136.520020  2308.709961  10572.019531  4012.909912  6031.859863   
2010-01-06  1137.140015  2301.090088  10573.679688  4017.669922  6034.330078   
2010-01-07  1141.689941  2300.050049  10606.860352  4024.800049  6019.359863   
2010-01-08  1144.979980  2317.169922  10618.190430  4045.139893  6037.609863   

                FTSE100     Nikkei225      HangSeng  MSCIWorld  
Date                                                            
2010-01-04  5500.299805  10654.790039  21823.279297        NaN  
2010-01-05  5522.500000  10681.830078  22279.580078        NaN  
2010-01-06  5530.000000  10731.450195  22416.669922        NaN  
2010-01-07  5526.700195  10681.660156  22269.449219        NaN  
2010-01-08  5534.200195  10798.320312  22296.7500

In [16]:
# Major Government Bond ETFs and Yield Proxies
bond_tickers = {
    "US10Y": "^TNX",       # 10-Year Treasury Yield (%)
    "US2Y": "^IRX",        # 2-Year Treasury Yield (%)
    "TLT": "TLT",          # 20+ Year Treasury ETF
    "IEF": "IEF",          # 7-10 Year Treasury ETF
    "BND": "BND",          # Total US Bond Market ETF
    "LQD": "LQD"           # Investment Grade Corporate Bond ETF
}

bonds = fetch_data(list(bond_tickers.values()))
bonds.columns = bond_tickers.keys()
bonds.head()

,US10Y,US2Y,TLT,IEF,BND,LQD
Date,,,,,,
2010-01-04,3.841,0.055,56.984020,62.541996,49.459587,58.760689
2010-01-05,3.755,0.060,57.352081,62.816563,49.604176,59.041294
2010-01-06,3.808,0.045,56.584351,62.563110,49.585281,58.867302
2010-01-07,3.822,0.045,56.679508,62.563110,49.547596,58.940304
2010-01-08,3.808,0.040,56.654148,62.640591,49.597874,59.069366


In [17]:
# Leading Stocks by Sector
stock_tickers = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Google": "GOOGL",
    "Amazon": "AMZN",
    "Meta": "META",
}

stocks = fetch_data(list(stock_tickers.values()))
stocks.columns = stock_tickers.keys()
stocks.head()

,Apple,Microsoft,Google,Amazon,Meta
Date,,,,,
2010-01-04,6.418384,23.130051,15.576998,6.6950,NaN
2010-01-05,6.429481,23.137535,15.508403,6.7345,NaN
2010-01-06,6.327212,22.995533,15.117455,6.6125,NaN
2010-01-07,6.315513,22.756386,14.765528,6.5000,NaN
2010-01-08,6.357501,22.913334,14.962370,6.6760,NaN


In [18]:
# Merge all classes on Date
market_data = (
    indexes
    .join(bonds, how="outer")
    .join(stocks, how="outer")
)

# Fill and clean
market_data = market_data.sort_index().bfill() #fillna(method="bfill")
market_data.head(10)
market_data.info(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4145 entries, 2010-01-04 to 2025-12-05
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S&P500     4144 non-null   float64
 1   NASDAQ     4145 non-null   float64
 2   DowJones   4144 non-null   float64
 3   CAC40      4145 non-null   float64
 4   DAX        4145 non-null   float64
 5   FTSE100    4145 non-null   float64
 6   Nikkei225  4145 non-null   float64
 7   HangSeng   4145 non-null   float64
 8   MSCIWorld  4145 non-null   float64
 9   US10Y      4145 non-null   float64
 10  US2Y       4145 non-null   float64
 11  TLT        4145 non-null   float64
 12  IEF        4145 non-null   float64
 13  BND        4145 non-null   float64
 14  LQD        4145 non-null   float64
 15  Apple      4145 non-null   float64
 16  Microsoft  4145 non-null   float64
 17  Google     4145 non-null   float64
 18  Amazon     4145 non-null   float64
 19  Meta       4145 non-null   flo

In [19]:
# Save files
market_data.to_csv(OUT / "Market_Prices_All.csv")
print(f"Saved market CSV to: {OUT.resolve()}")

Saved market CSV to: C:\Users\dax_a\Documents\GitHub\ESILV-MLproject-AU-BEJOT\data


In [20]:
returns = market_data.pct_change(fill_method=None).dropna()
returns.to_csv(OUT / "Market_Returns_All.csv")

In [21]:
# ===============================================================
# MERGING DATASETS
# ===============================================================

from pathlib import Path

OUT = Path("../data")
OUT.mkdir(parents=True, exist_ok=True)

market = pd.read_csv(OUT / 'Market_Returns_All.csv', index_col=0, parse_dates=True)
macro = pd.read_csv(OUT / 'Macro_AllSources.csv', index_col=0, parse_dates=True)

In [22]:
# Merge market and macroeconomic data
data = market.join(macro, how='left').ffill()
print("OK Combined dataset shape:", data.shape)
data.tail(5)
data.to_csv(OUT / "Full_dataset_reference.csv")

OK Combined dataset shape: (4143, 37)


In [23]:
# ===============================================================
# CREATE BINARY TARGET (Direction)
# ===============================================================

target_index = "Apple" if "Apple" in data.columns else data.columns[0]

# Create returns and technical indicators
data["Return"] = data[target_index] #.pct_change()
data["Volatility_20d"] = data["Return"].rolling(20).std()
data["MA20"] = data[target_index].rolling(20).mean()
data["MA50"] = data[target_index].rolling(50).mean()
data["Momentum"] = data[target_index] / data[target_index].shift(5) - 1

# Relative Strength Index (RSI)
def compute_RSI(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

data["RSI"] = compute_RSI(data[target_index])

# Define binary target variable: 1 = Up, 0 = Down
data["Direction"] = (data[target_index].shift(-1) > 0).astype(int)

# Drop rows with NaN
data = data.dropna()
print("OK Final dataset after feature engineering:", data.shape)

data.tail(20)

OK Final dataset after feature engineering: (4091, 44)


,S&P500,NASDAQ,DowJones,CAC40,DAX,FTSE100,Nikkei225,HangSeng,MSCIWorld,US10Y,...,Public_Debt_GDP_Pct,OECD_CPI_2015idx_USA,OECD_Unemp_rate_pct_USA,Return,Volatility_20d,MA20,MA50,Momentum,RSI,Direction
Date,,,,,,,,,,,,,,,,,,,,,
2025-11-07,0.001262,-0.002145,0.001594,-0.001832,-0.006912,-0.005464,-0.011935,-0.009215,0.002141,0.000000,...,114.755553,136.439499,4.3,-0.004819,0.012141,0.004598,0.003025,0.269771,35.348703,1
2025-11-10,0.015401,0.022719,0.008120,0.013249,0.016548,0.010803,0.012638,0.015518,0.014297,0.004153,...,114.755553,136.439499,4.3,0.004549,0.012080,0.004338,0.003324,-1.931684,51.027615,1
2025-11-11,0.002075,-0.002502,0.011808,0.012503,0.005345,0.011484,-0.001352,0.001777,0.003186,0.002433,...,114.755553,136.439499,4.3,0.021601,0.012635,0.005396,0.003756,4.870486,65.644849,0
2025-11-12,0.000630,-0.002635,0.006820,0.010423,0.012180,0.001192,0.004335,0.008478,0.002584,-0.013349,...,114.755553,136.439499,4.3,-0.006467,0.012906,0.004756,0.002865,-18.461073,45.793432,0
2025-11-13,-0.016557,-0.022904,-0.016529,-0.001062,-0.013939,-0.010463,0.004279,0.005583,-0.016162,0.011562,...,114.755553,136.439499,4.3,-0.001901,0.012681,0.005040,0.002717,0.388196,44.259951,0
2025-11-14,-0.000502,0.001322,-0.006527,-0.007580,-0.006866,-0.011144,-0.017653,-0.018490,-0.000218,0.008755,...,114.755553,136.439499,4.3,-0.001978,0.012292,0.003963,0.002685,-0.589447,39.235969,0
2025-11-17,-0.009162,-0.008407,-0.011819,-0.006251,-0.011980,-0.002372,-0.001045,-0.007082,-0.010426,-0.003616,...,114.755553,136.439499,4.3,-0.018171,0.010095,0.001083,0.002473,-4.994826,41.353485,0
2025-11-18,-0.008256,-0.012120,-0.010700,-0.018609,-0.017379,-0.012723,-0.032210,-0.017217,-0.008330,-0.002419,...,114.755553,136.439499,4.3,-0.000075,0.010095,0.000978,0.002768,-1.003460,48.932315,1
2025-11-19,0.003754,0.005857,0.001020,-0.001777,-0.000760,-0.004700,-0.003394,-0.003833,0.001613,0.002425,...,114.755553,136.439499,4.3,0.004188,0.009240,0.002009,0.003497,-1.647586,49.160080,0


In [24]:
data.to_csv(OUT / "Full_dataset_reference.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4091 entries, 2010-03-15 to 2025-12-04
Data columns (total 44 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   S&P500                             4091 non-null   float64
 1   NASDAQ                             4091 non-null   float64
 2   DowJones                           4091 non-null   float64
 3   CAC40                              4091 non-null   float64
 4   DAX                                4091 non-null   float64
 5   FTSE100                            4091 non-null   float64
 6   Nikkei225                          4091 non-null   float64
 7   HangSeng                           4091 non-null   float64
 8   MSCIWorld                          4091 non-null   float64
 9   US10Y                              4091 non-null   float64
 10  US2Y                               4091 non-null   float64
 11  TLT                                409